### Quelle: https://www.kaggle.com/code/enwei26/mnist-digits-pytorch-cnn-99/notebook

In [ ]:
import numpy as np
import pandas as pd

# Daten zum Download:
# https://www.kaggle.com/c/digit-recognizer
# https://www.kaggle.com/code/jagdish2386/fashion-mnist-dnn/data

# Das hier für Google Colab:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/My\ Drive/Colab\ Notebooks

import os
filesInput = os.listdir("./input")
print( f"files found in input dir: {filesInput}" )

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, utils
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print( f"device: {device}" )

### Load Data

In [ ]:
# MNIST
#df_train = pd.read_csv('./input/train.csv')
#df_test = pd.read_csv('./input/test.csv')

# Fashion MNIST
df_train = pd.read_csv('./input/fashion-mnist_train.csv')
df_test = pd.read_csv('./input/fashion-mnist_test.csv')

print('Train size: ', df_train.shape)
print('Test size: ', df_test.shape)
#df_train.head()

### Calculate mean and std of training data - used for normalization later

In [ ]:
train_data = df_train.drop('label', axis=1).values
train_mean = train_data.mean()/255.
train_std = train_data.std()/255.
print( f"Mean: {train_mean}  StdDev: {train_std}" )

### Split training data into training-validation

In [ ]:
# Train-Val split
mask = np.random.rand(len(df_train)) < 0.8
df_val = df_train[~mask]
df_train = df_train[mask]
print('Train size: ', df_train.shape)
print('Val size: ', df_val.shape)
print('Test size: ', df_test.shape)
#df_train.head()

### Visualize Example

In [ ]:
import matplotlib.pyplot as plt
ind = np.random.randint(0, df_train.shape[0]-1)
plt.imshow(df_train.iloc[ind].values[1:].reshape((28,28)), cmap='gray')
plt.title(str(df_train.iloc[ind][0]))

### Define a PyTorch Dataset

In [ ]:
# Create dataset class for PyTorch
class MNISTDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, n):
        data = self.df.iloc[n]
        image = data[1:].values.reshape((28,28)).astype(np.uint8)
        label = data[0]
        if self.transform:
            image = self.transform(image)
        return (image, label)

### Define data augmentation and data loaders

In [ ]:
# Initialize transformation, datasets, and loaders
batch_size = 32
classes = range(10)
train_transform = transforms.Compose(
                    [
                    transforms.ToPILImage(),
                    transforms.RandomAffine(degrees=(-10,10), translate=(0.1,0.1), scale=(0.95, 1.05)),
                    transforms.ColorJitter(brightness=0.1, contrast=0.1),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[train_mean], std=[train_std]),
                    ])
# don't (really) need the data augmentation in validation
val_transform = transforms.Compose(
                    [
                    transforms.ToPILImage(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[train_mean], std=[train_std]),
                    ])
test_transform = val_transform

train_dataset = MNISTDataset(df_train, transform = train_transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = MNISTDataset(df_val, transform = val_transform)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

### Sanity check to make sure data is of normal distribution (zero mean and unit standard dev)

In [ ]:
# sanity check for training data
imgs, lbls = next(iter(train_loader))
imgs[7].data.shape
print(imgs.data.min())
print(imgs.data.max())
print(imgs.data.mean())
print(imgs.data.std())
print(classes[lbls[0]])
plt.imshow(imgs[0].data.reshape((28,28)), cmap="gray")

In [ ]:
# sanity check for validation data
imgs, lbls = next(iter(val_loader))
imgs[0].data.shape
print(imgs.data.min())
print(imgs.data.max())
print(imgs.data.mean())
print(imgs.data.std())
print(classes[lbls[0]])
plt.imshow(imgs[0].data.reshape((28,28)), cmap="gray")

### Define network architecture

In [ ]:
# model definition
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.hidden1 = nn.Sequential(
            nn.Linear(28 * 28, 100),
            nn.Sigmoid()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(100, 10)
        )
                
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.hidden1(x)     
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

### Model training

In [ ]:
from torch.optim import SGD

model = Model()
model.to(device)
criterion = nn.NLLLoss()   # with log_softmax() as the last layer, this is equivalent to cross entropy loss
optimizer = SGD(model.parameters(), lr=1e-2)

model

In [ ]:
# Training Time!
import time
import copy
import matplotlib.pyplot as plt

# Some initialization work first...
epochs = 500
train_losses, val_losses = [], []
train_accu, val_accu = [], []
start_time = time.time()
early_stop_counter = 10   # stop when the validation loss does not improve for 10 iterations to prevent overfitting
counter = 0
best_val_loss = float('Inf')

for e in range(epochs):
    epoch_start_time = time.time()
    running_loss = 0
    accuracy=0
    # training step
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        log_ps = model(images)
        
        ps = torch.exp(log_ps)                
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # record training loss and error, then evaluate using validation data
    train_losses.append(running_loss/len(train_loader))
    train_accu.append(accuracy/len(train_loader))
    val_loss = 0
    accuracy=0
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            log_ps = model(images)
            val_loss += criterion(log_ps, labels)

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))
    val_losses.append(val_loss/len(val_loader))
    val_accu.append(accuracy/len(val_loader))

    print("Epoch: {}/{} ".format(e+1, epochs),
          "Time: {:.2f}s ".format(time.time()-epoch_start_time),
          "Training Loss: {:.3f} ".format(train_losses[-1]),
          "Training Accu: {:.2f}% ".format(100*train_accu[-1]),
          "Val Loss: {:.3f} ".format(val_losses[-1]),
          "Val Accu: {:.2f}%".format(100*val_accu[-1]))

    print( '  Epoch took %6.2fs, toal %8.2fs' % (time.time()-epoch_start_time, time.time()-start_time) )
    
    if val_losses[-1] < best_val_loss:
        best_val_loss = val_losses[-1]
        counter=0
        best_model_wts = copy.deepcopy(model.state_dict())
    else:
        counter+=1
        print('Validation loss has not improved since: {:.3f}..'.format(best_val_loss), 'Count: ', str(counter))
        if counter >= early_stop_counter:
            print('Early Stopping now!!')
            model.load_state_dict(best_model_wts)
            break

In [ ]:
# plot training history

plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
ax = plt.gca()
ax.set_xlim([0, e + 2])
plt.ylabel('Loss')
plt.plot(range(1, e + 2), torch.FloatTensor(train_losses[:e+1]), 'b', label='Training Loss')
plt.plot(range(1, e + 2), torch.FloatTensor(val_losses[:e+1]), 'r', label='Validation Loss')
ax.grid(linestyle='-.')
plt.legend()
plt.subplot(2,1,2)
ax = plt.gca()
ax.set_xlim([0, e+2])
plt.ylabel('Accuracy')
plt.plot(range(1, e + 2), torch.FloatTensor(train_accu[:e+1]), 'b', label='Training Accuracy')
plt.plot(range(1, e + 2), torch.FloatTensor(val_accu[:e+1]), 'r', label='Validation Accuracy')
ax.grid(linestyle='-.')
plt.legend()
plt.show()

### Prediction

In [ ]:
# prepare to predict test data - REMEMBER PRE-PROCESSING!
# I originally forgot to scale and normalize, which caused problems....

# some sanity check to make sure
x_test = df_test.values
x_test = x_test.reshape([-1, 28, 28]).astype(float)
x_test = x_test/255.
x_test = (x_test-train_mean)/train_std
print(x_test.min())
print(x_test.max())
print(x_test.mean())
print(x_test.std())

In [ ]:
x_test = np.expand_dims(x_test, axis=1)
x_test = torch.from_numpy(x_test).float().to(device)
x_test.type()

In [ ]:
# prediction time!
model.eval()   # this is needed to disable dropouts
with torch.no_grad():    # turn off gradient computation because we don't need it for prediction
    ps = model(x_test)
    prediction = torch.argmax(ps, 1)
    print('Prediction', prediction)

In [ ]:
# prepare output file
df_export = pd.DataFrame(prediction.cpu().tolist(), columns = ['Label'])
df_export['ImageId'] = df_export.index +1
df_export = df_export[['ImageId', 'Label']]
df_export.head()

In [ ]:
df_export.to_csv('output.csv', index=False)